### ResNetの実装
ResNetの基本式 $y = F(x) + x$
ここで $F(x)$ は残差関数

$$
\text{Output} = \text{ReLU}(F(x) + x)
$$

### BatchNormalization()の中身について
> ReLUにそのまま通してしまったら負数の情報が全て0になってしまうので、標準化を行う
> $$ \hat{x} = \frac{x - \mu (\text{平均})}{\sigma (\text{標準偏差})} $$
- 入ってきたデータ（ミニバッチ）に対して**標準化「平均を0,分散を1」**
- データから平均値を引くことで中心を0としている

In [ ]:
import tensorflow as tensorflow
from tensorflow.keras.layers import Input, Conv2D, Activation, Add,BatchNormalization

def residual_block(input_tensor, num_filters,strides=1):
    """
    入力と出力のサイズが変わらない、基本的な残差ブロック
    残差：Residual
    Output = f(Input)+Input
        ここでFは2回の畳み込みとReLU活性化

    Skip Connection
        残差分だけを学習することで、層が深くなっても勾配損失問題を防ぐ
    
    Conv
        畳み込みをすると通常は画面の端っこが削れて小さくなるが、padding='same'を指定して周りに余白をつける
        そうすると入力と出力のサイズが変わらなくなる

    Add()([x,shortcut])
        KerasのFunctional APIの書き方
            ➀足し算マシーン(Addレイヤー)の生成、ここでAddは単純に足し算をするだけなので()の中に複雑な設定が必要ない
            ➁足し算マシーンに[x,shortcut]というリストの形状にして足し算を実行する

    Strides
        畳み込みの移動幅を指定するパラメータ
        1なら通常通り1pxずつ、2なら2pxずつ移動する(1個飛ばし)ので、出力サイズが半分になる

    num_filters
        各num_filterは畳み込み層で検知した特徴マップを持っている
        例えば浅い層のフィルターではエッジ検出、深い層ではより複雑なパターンを検出する
            エッジ：隣り合う画素(ピクセル)が急激に変化している部分
    """
    shortcut = input_tensor

    # 畳み込み一回目
    x = Conv2D(num_filters, (3, 3), padding='same',strides=strides)(input_tensor)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)

    # 畳み込み二回目
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x= BatchNormalization()(x)

    if strides > 1 or input_tensor.shape[-1] != num_filters:
        """
        サイズが変わる、またはチャンネル数が変わるとき
        .shape[-1]は最後の要素、つまりchannnel
        """
        shortcut = Conv2D(num_filters, (1, 1), padding='same',strides=strides)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense


def build_simple_resnet(input_shape=(128, 128, 3), num_classes=10):
    """
    ResNetの簡易モデルの作成
        画像分類タスクでは、画像の特徴の身を知りたいので畳み込み層のエンコードしか必要ない
        セグメンテーションは位置情報も必要なのでdecodeも必要になる
        ResNet18の18は層の数、50や152などがある

    Projection(射影)
            ➀入力の特徴マップのサイズを変換する
            ➁フィルター数(channel数)を変更する

    GlobalAveragePooling2D
        Strides=1は画像サイズを変えないまま特徴をしっかり抽出する

    """
    inputs = Input(input_shape)
    x = residual_block(inputs, num_filters=64, strides=1)

    x = residual_block(
        x, num_filters=64, strides=1
    )  # サイズを変えないまま特徴をしっかり見極める
    x = residual_block(x, num_filters=64, strides=1)

    x = residual_block(
        x, num_filters=128, strides=2
    )  # サイズを半分にする(Projection Block)
    x = residual_block(x, num_filters=128, strides=1)

    # --- 出口 (Output Layers) ---
    x = GlobalAveragePooling2D()(x)

    # 最終判定 (10クラス分類)
    outputs = Dense(num_classes, activation="softmax")(x)

    return Model(inputs, outputs)

### 1. GlobalAveragePooling2D　(GAP)
**位置情報は捨てて、特徴の強さだけを残す層**
> 一枚の特徴マップの全画素の平均を計算して、有効な数字であればその特徴が存在すると考える
  - 入力: $(Batch, 8, 8, 128)$ → 高さ8, 幅8, チャンネル128
  - 出力: $(Batch, 128)$ → 長さ128のベクトル

### 2. Dense （全結合層）
  - 最後のDense層のユニット数は、**分類したいクラス数**（EuroSATなら10個）と一致させる
  - 最後に`Softmax`関数を通すことで、出力を確率（合計すると100%）に変換

In [ ]:
import tensorflow_datasets as tfds
import os

# データを保存するディレクトリ (先ほど作った data フォルダを指定)
DATA_DIR = "data"

"""
tfds.load()でdatasetをダウンロードして読みこむ
    タプルのデータセットとinfoを返す
        eurosatのRGBバージョンを使用する
        as_supervised=True で(画像, ラベル)のタプルで返す
        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'] で学習・検証・テストに分割

"""
(train_ds, val_ds, test_ds), info = tfds.load(
    "eurosat/rgb",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    data_dir=DATA_DIR,
    as_supervised=True,
    with_info=True,
)

print("ダウンロードと読み込みが完了しました！")
print(f"クラス名: {info.features['label'].names}")
print(f"学習データ数: {len(train_ds)}")
print(f"検証データ数: {len(val_ds)}")
print(f"テストデータ数: {len(test_ds)}")

ダウンロードと読み込みが完了しました！
クラス名: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
学習データ数: 21600
検証データ数: 2700
テストデータ数: 2700


In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

print(f"現在のポリシー: {policy.compute_dtype}")

現在のポリシー: float16


In [ ]:
import tensorflow as tf

# Setting constants
IMG_SIZE = 64
BATCH_SIZE = 32


def preprocess_data(image, label):
    """
    画像データの前処理を行う関数
        1. サイズを確実に合わせる (リサイズ)
            - `tf.image.resize()`はTensorflowのimageモジュールの中にある関数
            - interpolation 補間：いい感じに新しい色を計算して埋める
            > 有限要素法のバイリニア補間
                すべての画素に対して、周囲の整数格子点の値を使って線形補間を行う方法

        2. 正規化 (Normalization)
            - 画像のRGB各チャンネルの値を0から1の範囲にスケーリングする
                - 正規化しないと計算量が膨大になる

        .take(n)
            - データセットから最初のn個の要素を取得するメソッド
    """
    image = tf.image.resize(
        image, (IMG_SIZE, IMG_SIZE)
    )  # 1.サイズを確実に合わせる(リサイズ)
    image = tf.cast(image, tf.float32) / 255.0  # 2.正規化(Normalization)
    return image, label


# --- パイプラインの構築 ---
AUTOTUNE = tf.data.AUTOTUNE
train_batches = (
    train_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .shuffle(buffer_size=1000)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

val_batches = (
    val_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

# 確認
for img_batch, label_batch in train_batches.take(1):
    print(f"Image Batch Shape: {img_batch.shape}")
    print(f"Label Batch Shape: {label_batch.shape}")

    # Image: (32, 64, 64, 3) -> 32枚, 64x64ピクセル, 3チャンネル(RGB)
    # Label: (32,) -> 32個の正解ラベル

Image Batch Shape: (32, 64, 64, 3)
Label Batch Shape: (32,)


### データパイプライン DataPipeLine
> データパイプラインとは、データをストレージから読み出し、計算モデルが処理可能なテンソルへと変換し、GPUやTPUへ供給する一連の処理工程、およびそれを実装したソフトウェアアーキテクチャを指す。

In [ ]:
# input_shapeは (64, 64, 3), クラス数は 10
model = build_simple_resnet(input_shape=(64, 64, 3), num_classes=10)

# --- コンパイル (学習ルールの設定) ---
model.compile(
    # AIの「間違いの修正方法」を指定します。Adamは最も一般的で優秀な修正担当者です。
    optimizer="adam",  # Optimizer (最適化アルゴリズム): 'adam'
    # Loss (損失関数): 'SparseCategoricalCrossentropy'
    # from_logits=True は、AIの生の出力を確率に変換してから計算しろ、という指示です。
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # Metrics:'accuracy'
    metrics=["accuracy"],
)

# モデルの設計図確認
# model.summary()

In [ ]:
# --- 学習の実行 ---
history = model.fit(train_batches, validation_data=val_batches, epochs=20)

Epoch 1/5


KeyboardInterrupt: 

###　Result
- 自分のPCのCPU,Core i7だと90分かかった![Result of CPU Learning](images/122101.png)
<br>
- AI工房のGPUサーバーを使うと40秒程度で5epoch終了->**135倍!!のスピード**![Result of GPU Learning](images/122102.png)

In [ ]:
# --- テストデータの準備と評価 ---

# テストデータにも同様の前処理（リサイズ・正規化）とバッチ化を適用
test_batches = (
    test_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

# モデルの評価 (未学習データでの性能確認)
print("テストデータで評価を実行します...")
test_loss, test_acc = model.evaluate(test_batches)

print(f"\nテストデータの正解率: {test_acc * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# クラス名を取得
class_names = info.features["label"].names


def plot_predictions(dataset, model, num_images=9):
    """
    テストデータに対して推論を行い、画像と予測結果を表示する関数
    """
    plt.figure(figsize=(12, 12))

    # データセットから1バッチ(32枚)だけ取り出す
    for images, labels in dataset.take(1):
        # 推論の実行 (確率が出力される)
        predictions = model.predict(images)
        # 最も確率が高いクラスのインデックスを取得
        pred_indices = np.argmax(predictions, axis=1)

        # 指定枚数分だけ表示
        for i in range(min(num_images, len(images))):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i])

            # ラベル名の取得
            true_label = class_names[labels[i]]
            pred_label = class_names[pred_indices[i]]
            confidence = 100 * np.max(predictions[i])

            # 正解なら青、不正解なら赤でタイトルを表示
            color = "blue" if labels[i] == pred_indices[i] else "red"

            plt.title(
                f"Pred: {pred_label} ({confidence:.1f}%)\nTrue: {true_label}",
                color=color,
            )
            plt.axis("off")

    plt.tight_layout()
    plt.show()


# 可視化の実行
print("推論結果の可視化:")
plot_predictions(test_batches, model)

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm

# --- 分析用データの準備 ---
print("詳細なエラー分析を実行します...")

# 全テストデータの予測と正解ラベルの取得
all_images = []
all_labels = []

# データセットから全データを取得
for img_batch, label_batch in test_batches:
    all_images.append(img_batch.numpy())
    all_labels.append(label_batch.numpy())

x_test = np.concatenate(all_images)
y_test = np.concatenate(all_labels)

# 推論実行 (バッチ処理しているので高速)
predictions = model.predict(x_test, verbose=0)
pred_labels = np.argmax(predictions, axis=1)
max_probs = np.max(predictions, axis=1)

# --- 1. 混同行列 (Confusion Matrix) ---
conf_matrix = confusion_matrix(y_test, pred_labels)

# --- 2. 「自信満々に間違えた」データの抽出 ---
incorrect_indices = np.where(pred_labels != y_test)[0]
# 予測確率(自信)が高い順にソートしてトップを取得
sorted_incorrect_indices = incorrect_indices[
    np.argsort(max_probs[incorrect_indices])[::-1]
]


# --- 3. Grad-CAM (判断根拠の可視化) ---
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    if len(img_array.shape) == 3:
        img_array = np.expand_dims(img_array, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


def find_target_layer(model):
    for layer in reversed(model.layers):
        if len(layer.output_shape) == 4:
            return layer.name
    return None


target_layer_name = find_target_layer(model)

# --- 可視化プロット ---
plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 0.5, 0.5])

# 左側: 混同行列
ax_cm = plt.subplot(gs[:, :2])
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
    ax=ax_cm,
)
ax_cm.set_title("Confusion Matrix", fontsize=16)
ax_cm.set_ylabel("True Label", fontsize=14)
ax_cm.set_xlabel("Predicted Label", fontsize=14)

# 右側: 自信満々に間違えた画像 Top 4
num_display = 4
for i in range(min(num_display, len(sorted_incorrect_indices))):
    idx = sorted_incorrect_indices[i]
    img = x_test[idx]
    true_lb = class_names[y_test[idx]]
    pred_lb = class_names[pred_labels[idx]]
    conf = max_probs[idx]

    # Grad-CAM ヒートマップ
    heatmap = make_gradcam_heatmap(
        img, model, target_layer_name, pred_index=pred_labels[idx]
    )

    # リサイズと重ね合わせ表示
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (IMG_SIZE, IMG_SIZE))
    heatmap_resized = tf.squeeze(heatmap_resized).numpy()

    # グリッド配置 (右半分の領域を使う)
    row = i // 2
    col = 2 + (i % 2)
    ax_img = plt.subplot(gs[row, col])

    ax_img.imshow(img)
    ax_img.imshow(heatmap_resized, alpha=0.5, cmap="jet")

    # 枠線の色 (赤: 間違い)
    for spine in ax_img.spines.values():
        spine.set_edgecolor("red")
        spine.set_linewidth(3)

    ax_img.set_title(
        f"True: {true_lb}\nPred: {pred_lb}\nConf: {conf:.1%}",
        fontsize=12,
        color="red",
        fontweight="bold",
    )
    ax_img.set_xticks([])
    ax_img.set_yticks([])

plt.suptitle(
    f"Error Analysis & Grad-CAM (Target Layer: {target_layer_name})", fontsize=20
)
plt.tight_layout()
plt.show()

# 考察コメントの例示
print("\n=== 考察コメントの例 ===")
print(
    "Grad-CAMのヒートマップ（赤色部分）を確認することで、モデルが画像のどこを見て判断したかが分かります。"
)
print(
    "これを用いて、『形ではなく色だけで判断してしまっている』などの誤答原因を分析できます。"
)